In [ ]:
%%spark

VBox()

Starting Spark application


In [17]:
input_data = "s3a://udacity-dend/"
output_data = "s3://udacity-spark-project/output/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
# get filepath to song data file
song_data = input_data + 'song_data/*/*/*/*.json'

# read song data file
df = spark.read.json(song_data)

# extract columns to create songs table
df.createOrReplaceTempView("songs_table")
songs_table = spark.sql("""
select 
    song_id, 
    title, 
    artist_id, 
    year, 
    duration
from songs_table
where song_id is not null
""")

# write songs table to parquet files partitioned by year and artist
#     songs_table.write.mode('overwrite').partitionBy('year', 'artist_id').parquet(output_data+'songs_table/')

# extract columns to create artists table
artists_table = spark.sql("""
select 
    artist_id,
    artist_name as name,
    artist_location as location,
    artist_latitude as latitude,
    artist_longitude as longitude
from songs_table
where artist_id is not null
""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# get filepath to log data file
log_data = input_data + 'log_data/*/*/*.json'

# read log data file
df1 = spark.read.json(log_data)

# filter by actions for song plays
df1.filter(df1.page == 'NextSong')
df1.createOrReplaceTempView('log_table')

# extract columns for users table    
users_table = spark.sql("""
select
    userId as user_id, 
    firstName as first_name, 
    lastName as last_name, 
    gender, 
    level
from log_table
where userId is not null
""")


# extract columns to create time table
time_table = spark.sql("""
select 
    start_time, 
    hour(start_time) as hour, 
    dayofmonth(start_time) as day, 
    weekofyear(start_time) as week, 
    month(start_time) as month, 
    year(start_time) as year, 
    dayofweek(start_time) as weekday
from
(
    select 
        to_timestamp(ts/1000) as start_time
    from log_table
    where ts is not null
) as a
""")


# extract columns from joined song and log datasets to create songplays table 
songplays_table = spark.sql("""
select 
    monotonically_increasing_id() as songplay_id, 
    a.start_time, 
    month(a.start_time) as month,
    year(a.start_time) as year,
    a.user_id, 
    a.level, 
    b.song_id, 
    b.artist_id, 
    a.session_id, 
    a.location, 
    a.user_agent
from 
(
    select 
        to_timestamp(ts/1000) as start_time,
        userId as user_id,
        level,
        sessionId as session_id,
        location,
        userAgent as user_agent,
        song
    from log_table
) as a
inner join
(
    select
        song_id,
        artist_id,
        title
    from 
        songs_table
) as b
on a.song = b.title
""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
spark.sql("""
select
    gender,
    count(userId)
from log_table
where userId is not null
group by gender
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------------+
|gender|count(userId)|
+------+-------------+
|  null|          286|
|     M|         2288|
|     F|         5482|
+------+-------------+

In [ ]:
songplays_table.createOrReplaceTempView('songsplay')
spark.sql("""
select year, month, count(songplay_id) from songsplay group by year, month
""").show()